# DeepGONet - interpretation tool

**Objective:** this tool points out the main biological functions used for cancer predictions and quantify their  contribution via LRP relevance score. It can be used at three levels: disease, subdisease, and patient.

To be used, the model has to be trained first and saved with the script *DeepGONet.py* (flag ``save=True``).
Moreover, the package [*innvestigate*](https://github.com/albermax/innvestigate) has to be installed beforehand.

#### Load packages

In [1]:
import warnings
import os

In [2]:
import scipy
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import seaborn
from sklearn import preprocessing
import gc
from tqdm import tqdm
from tqdm import tqdm_notebook
from sklearn.utils import class_weight
import matplotlib.backends.backend_pdf

/nhome/siniac/vbourgeais/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/nhome/siniac/vbourgeais/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/nhome/siniac/vbourgeais/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/nhome/siniac/vbourgeais/venv/lib/python3.6/site-packages/ten

In [3]:
from keras.optimizers import Adam,SGD,Nadam
from sklearn.utils import class_weight
from keras.utils import to_categorical, plot_model
from keras.models import Model
from keras import backend as K
from keras import regularizers
from keras.layers import Layer
from keras.layers import Input, Dense, BatchNormalization, Dropout, ReLU, Activation,Lambda
from keras.models import Model
from keras.callbacks import ReduceLROnPlateau
from keras.backend import clear_session, set_session,l2_normalize
from keras.utils import multi_gpu_model

Using TensorFlow backend.


In [4]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

['/job:localhost/replica:0/task:0/device:GPU:0']

#### Environnement definition

In [5]:
warnings.filterwarnings("ignore")
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0" #your GPU device
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

In [9]:
dir_data = #TO BE DEFINED
save_dir= #TO BE DEFINED

## 1. Definition of the network

### a) Definition of the variables of the network

- number of layers/neurons by layer

In [6]:
NB_LAYERS = 6
N_HIDDEN_6 = 90 #Level 2
N_HIDDEN_5 = 255 #Level 3
N_HIDDEN_4 = 515 #Level 4
N_HIDDEN_3 = 951 #Level 5
N_HIDDEN_2 = 1386 #Level 6
N_HIDDEN_1 = 1574 #Level 7
SHAPE_FEATURES = 54675 #Input Layer
SHAPE_LABEL = 1

- control of the regularization term $L_{GO}$

In [7]:
ALPHA = 1e-2 #fixe

- training hyperparameters

In [8]:
OPT = "adam"
LR = 1e-03
DROP_RATIO = 1.0

EPOCHS = 600
BATCH_SIZE = 2**9 #parameter that will be used later

NB_STACK = 1
BN = False

### b) Definition of the functions

In [10]:
def mlp_for_sigmoid(drop_ratio=0.60,trainable=True):
    
    inputs = Input(shape=(SHAPE_FEATURES,))
    
    first_layer = Dense(N_HIDDEN_1,name='first_layer')(inputs)   
    x = ReLU()(first_layer)        
    x = Dropout(rate=drop_ratio)(x)
    
    second_layer = Dense(N_HIDDEN_2,name='second_layer')(x)    
    x = ReLU()(second_layer)
    x = Dropout(rate=drop_ratio)(x)
    
    third_layer = Dense(N_HIDDEN_3,name='third_layer')(x)    
    x = ReLU()(third_layer)   
    x = Dropout(rate=drop_ratio)(x)   
    
    fourth_layer = Dense(N_HIDDEN_4,name='fourth_layer')(x)    
    x = ReLU()(fourth_layer)   
    x = Dropout(rate=drop_ratio)(x)
    
    fifth_layer = Dense(N_HIDDEN_5,name='fifth_layer')(x)
    x = ReLU()(fifth_layer)
    x = Dropout(rate=drop_ratio)(x)
    
    sixth_layer = Dense(N_HIDDEN_6,name='sixth_layer')(x)
    x = ReLU()(sixth_layer)
    x = Dropout(rate=drop_ratio)(x)
    
    last_layer = Dense(SHAPE_LABEL,name='last_layer')(x)    
    predictions = Activation('sigmoid')(last_layer)
    
    model = Model(inputs=inputs, outputs=predictions)
    
    return model

## 2. Evaluation of the model

### a) Load the data

In [11]:
%%time
loaded = np.load(os.path.join(dir_data,"X_test.npz"))
X_test = loaded['x']
y_test = loaded['y']
if SHAPE_LABEL>=2: 
    y_test=to_categorical(y_test)

CPU times: user 7.97 s, sys: 206 ms, total: 8.18 s
Wall time: 17.5 s


In [12]:
X_test.shape

(4462, 54675)

In [13]:
y_test.shape

(4462, 1)

### b) Restoration of the model learned with TF

#### i. Prepare the file "model_vars.txt"
It has to be executed only once. It means that if the notebook is executed again later, you can skip this step. You can directly jump to the step below.

In [ ]:
%%time
PATH_REL_META = os.path.join(save_dir,'model.meta')

#start tensorflow session
with tf.Session() as sess:
    #import graph
    saver = tf.train.import_meta_graph(PATH_REL_META)
    #load weights for graph
    saver.restore(sess, os.path.join(save_dir,"model"))

    #get all global variables (including model variables)
    vars_global = tf.global_variables()

    #get their name and value and put them into dictionary
    model_vars = {}
    
    with tf.device('/gpu:0'): #TO BE CHANGED if you didn't use this one.
        
        for var in vars_global:
            try:
                model_vars[var.name] = sess.run(var)
            except:
                print("For var={}, an exception occurred".format(var.name))

In [ ]:
with open(os.path.join(save_dir,"model_vars.txt"), "wb") as fp:
    #Pickling
    pickle.dump(model_vars, fp)

In [ ]:
tf.reset_default_graph()

#### ii. Fix the value of the model parameters (weights and biases)

In [14]:
indices = ['first_layer','second_layer','third_layer','fourth_layer',
                        'fifth_layer','sixth_layer','last_layer']

In [15]:
with open(os.path.join(save_dir,"model_vars.txt"), "rb") as fp:
    model_vars=pickle.load(fp)

In [16]:
model_for_innvestigate = mlp_for_sigmoid(drop_ratio=DROP_RATIO,trainable=False)

In [17]:
%%time
for layer_idx,layer_name in enumerate(indices):
    if layer_name=="last_layer":
        layer_idx="_out"
    else:
        layer_idx+=1
    patternW='W{}:0'.format(layer_idx)
    patternB='B{}:0'.format(layer_idx)
    model_for_innvestigate.get_layer(name=layer_name).set_weights([model_vars[patternW], model_vars[patternB]])

CPU times: user 299 ms, sys: 391 ms, total: 690 ms
Wall time: 1.06 s


#### iii. Launch the model on the test set

In [18]:
adam = Adam(lr=LR)
model_for_innvestigate.compile(optimizer=adam,loss='binary_crossentropy',metrics=['accuracy'])
results = model_for_innvestigate.evaluate(X_test, y_test)
print('test loss, test acc:', results)

4462/4462 [==============================] - 2s 440us/step
test loss, test acc: [0.1820492276450989, 0.924697445091887]


In [19]:
y_result = model_for_innvestigate.predict(X_test)

In [20]:
y_result_final = (y_result>0.5).astype(dtype=int)

In [21]:
y_result_final=y_result_final.reshape(-1)

In [22]:
y_result_final

array([0, 1, 1, ..., 1, 1, 1])

In [23]:
y_result_final[y_result_final==1].shape

(2950,)

In [24]:
y_result_final[y_result_final==0].shape

(1512,)

In [25]:
from sklearn.metrics import confusion_matrix,classification_report
print(confusion_matrix(y_test,y_result_final))
print(classification_report(y_test,y_result_final))

[[1344  168]
 [ 168 2782]]
              precision    recall  f1-score   support

         0.0       0.89      0.89      0.89      1512
         1.0       0.94      0.94      0.94      2950

   micro avg       0.92      0.92      0.92      4462
   macro avg       0.92      0.92      0.92      4462
weighted avg       0.92      0.92      0.92      4462



## 3. Biological Interpretation

In [26]:
%%time
matrix_connection = []
matrix_connection.append(pd.read_csv(os.path.join(dir_data,"matrix_connexion_h1.csv"),index_col=0))
matrix_connection.append(pd.read_csv(os.path.join(dir_data,"matrix_connexion_h2.csv"),index_col=0))
matrix_connection.append(pd.read_csv(os.path.join(dir_data,"matrix_connexion_h3.csv"),index_col=0))
matrix_connection.append(pd.read_csv(os.path.join(dir_data,"matrix_connexion_h4.csv"),index_col=0))
matrix_connection.append(pd.read_csv(os.path.join(dir_data,"matrix_connexion_h5.csv"),index_col=0))
matrix_connection.append(pd.read_csv(os.path.join(dir_data,"matrix_connexion_h6.csv"),index_col=0))

CPU times: user 10.5 s, sys: 501 ms, total: 11 s
Wall time: 51.7 s


In [27]:
import innvestigate
import innvestigate.utils as iutils

In [28]:
model_wo_sigmoid = Model(model_for_innvestigate.input, model_for_innvestigate.layers[len(model_for_innvestigate.layers)-2].output)

In [29]:
analyzer = innvestigate.create_analyzer("lrp.epsilon_IB",
                                        model_wo_sigmoid,
                                        reverse_keep_tensors=True,epsilon=1e-07)

### a) Use-case 1: Patient level

In [31]:
idx_sample = #TO BE DEFINED
nb_go_to_show = 5

Check the real, predicted, and probability class associated to this sample

In [54]:
y_test[idx_sample]

array([1.], dtype=float32)

In [55]:
y_result_final[idx_sample]

1

In [56]:
y_result[idx_sample]

array([0.9999999], dtype=float32)

After our model predicts the outcome of a patient with a probability score, the LRP relevance of each neuron is computed. 

In [52]:
analysis = analyzer.analyze(X_test)

In [53]:
LRP_matrix = analyzer._reversed_tensors

#### Interpretation of the prediction 

The neurons are sorted according to their relevance score and the most important ones are returned with their corresponding GO term and biological function.

##### First layer

In [57]:
idx_layer = 0

In [58]:
indexes = np.argsort(LRP_matrix[4][1][idx_sample])[::-1][:nb_go_to_show]
lrp_scores = np.sort(LRP_matrix[4][1][idx_sample])[::-1][:nb_go_to_show]

In [59]:
list_go=dict()
for idx,idx_go in enumerate(indexes[:nb_go_to_show]):
    list_go[matrix_connection[idx_layer].columns[idx_go]]= lrp_scores[idx]

Top-5

In [63]:
for go, lrp_score in list_go.items():
    print(go,":",np.round(lrp_score,decimals=2))

GO:0015031 : 1.83
GO:0006468 : 1.11
GO:0030335 : 1.09
GO:0007268 : 0.87
GO:0006412 : 0.86


The biological function associated to each GO term can be obtained with public tool like [QuickGO](https://www.ebi.ac.uk/QuickGO/).

##### Second layer

In [64]:
idx_layer += 1

In [65]:
indexes = np.argsort(LRP_matrix[7][1][idx_sample])[::-1][:nb_go_to_show]
lrp_scores = np.sort(LRP_matrix[7][1][idx_sample])[::-1][:nb_go_to_show]

In [66]:
list_go=dict()
for idx,idx_go in enumerate(indexes[:nb_go_to_show]):
    list_go[matrix_connection[idx_layer].columns[idx_go]]= lrp_scores[idx]

Top-5

In [67]:
for go, lrp_score in list_go.items():
    print(go,":",np.round(lrp_score,decimals=2))

GO:0044257 : 0.61
GO:0071420 : 0.56
GO:0043009 : 0.54
GO:1901258 : 0.53
GO:0010737 : 0.49


##### Third layer

In [68]:
idx_layer += 1

In [73]:
idx_layer

2

In [75]:
indexes = np.argsort(LRP_matrix[10][1][idx_sample])[::-1][:nb_go_to_show]
lrp_scores = np.sort(LRP_matrix[10][1][idx_sample])[::-1][:nb_go_to_show]

In [76]:
list_go=dict()
for idx,idx_go in enumerate(indexes[:nb_go_to_show]):
    list_go[matrix_connection[idx_layer].columns[idx_go]]= lrp_scores[idx]

Top-5

In [77]:
for go, lrp_score in list_go.items():
    print(go,":",np.round(lrp_score,decimals=2))

GO:1901355 : 0.49
GO:0035556 : 0.41
GO:0048382 : 0.36
GO:1905144 : 0.31
GO:0048864 : 0.29


##### Fourth layer

In [78]:
idx_layer += 1

In [79]:
indexes = np.argsort(LRP_matrix[13][1][idx_sample])[::-1][:nb_go_to_show]
lrp_scores = np.sort(LRP_matrix[13][1][idx_sample])[::-1][:nb_go_to_show]

In [80]:
list_go=dict()
for idx,idx_go in enumerate(indexes[:nb_go_to_show]):
    list_go[matrix_connection[idx_layer].columns[idx_go]]= lrp_scores[idx]

Top-5

In [81]:
for go, lrp_score in list_go.items():
    print(go,":",np.round(lrp_score,decimals=2))

GO:0071709 : 0.54
GO:0055085 : 0.43
GO:0014070 : 0.39
GO:0042127 : 0.38
GO:0010243 : 0.38


##### Fifth layer

In [82]:
idx_layer += 1

In [83]:
indexes = np.argsort(LRP_matrix[16][1][idx_sample])[::-1][:nb_go_to_show]
lrp_scores = np.sort(LRP_matrix[16][1][idx_sample])[::-1][:nb_go_to_show]

In [84]:
list_go=dict()
for idx,idx_go in enumerate(indexes[:nb_go_to_show]):
    list_go[matrix_connection[idx_layer].columns[idx_go]]= lrp_scores[idx]

Top-5

In [85]:
for go, lrp_score in list_go.items():
    print(go,":",np.round(lrp_score,decimals=2))

GO:0044091 : 0.62
GO:0060322 : 0.61
GO:0050794 : 0.57
GO:0050808 : 0.5
GO:0051098 : 0.5


##### Sixth layer

In [86]:
idx_layer += 1

In [87]:
indexes = np.argsort(LRP_matrix[19][1][idx_sample])[::-1][:nb_go_to_show]
lrp_scores = np.sort(LRP_matrix[19][1][idx_sample])[::-1][:nb_go_to_show]

In [88]:
list_go=dict()
for idx,idx_go in enumerate(indexes[:nb_go_to_show]):
    list_go[matrix_connection[idx_layer].columns[idx_go]]= lrp_scores[idx]

Top-5

In [89]:
for go, lrp_score in list_go.items():
    print(go,":",np.round(lrp_score,decimals=2))

GO:0010463 : 0.7
GO:0030534 : 0.7
GO:0006739 : 0.63
GO:0031128 : 0.61
GO:0048856 : 0.61
